In [2]:
import datasets
from evaluate import load

c:\Users\James\anaconda3\envs\cmu_10715_applied\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Setup

Stolen from docs
- https://huggingface.co/datasets/narrativeqa

In [3]:
ds = datasets.load_dataset("narrativeqa", split="validation")

Found cached dataset narrativeqa (C:/Users/James/.cache/huggingface/datasets/narrativeqa/default/0.0.0/daef7ccc51ec258bef464658d11751bb20f033da9b4c219fd84563b3a4af0422)


In [4]:
print(ds[0]["document"]["summary"]["text"])

 The play begins with three pages disputing over the black cloak usually worn by the actor who delivers the prologue. They draw lots for the cloak, and one of the losers, Anaides, starts telling the audience what happens in the play to come; the others try to suppress him, interrupting him and putting their hands over his mouth. Soon they are fighting over the cloak and criticizing the author and the spectators as well.
In the play proper, the goddess Diana, also called Cynthia, has ordained a "solemn revels" in the valley of Gargaphie in Greece. The gods Cupid and Mercury appear, and they too start to argue. Mercury has awakened Echo, who weeps for Narcissus, and states that a drink from Narcissus's spring causes the drinkers to "Grow dotingly enamored of themselves." The courtiers and ladies assembled for the Cynthia's revels all drink from the spring.
Asotus, a foolish spendthrift who longs to become a courtier and a master of fashion and manners, also drinks from the spring; embold

# Model

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [5]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base").to("cuda")

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
c:\Users\James\anaconda3\envs\cmu_10715_applied\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\James\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrato

In [6]:
inp = tokenizer(ds[0]["question"]["text"], ds[0]["document"]["summary"]["text"], return_tensors="pt").to("cuda")

In [19]:
print("Q", ds[0]["question"]["text"])
print("T", ds[0]["document"]["summary"]["text"])
print("A", ds[0]["answers"][0]["text"])

Q WHO NORMALLY DELIVERS THE OPENING PROLOGUE IN THE PLAY?
T  The play begins with three pages disputing over the black cloak usually worn by the actor who delivers the prologue. They draw lots for the cloak, and one of the losers, Anaides, starts telling the audience what happens in the play to come; the others try to suppress him, interrupting him and putting their hands over his mouth. Soon they are fighting over the cloak and criticizing the author and the spectators as well.
In the play proper, the goddess Diana, also called Cynthia, has ordained a "solemn revels" in the valley of Gargaphie in Greece. The gods Cupid and Mercury appear, and they too start to argue. Mercury has awakened Echo, who weeps for Narcissus, and states that a drink from Narcissus's spring causes the drinkers to "Grow dotingly enamored of themselves." The courtiers and ladies assembled for the Cynthia's revels all drink from the spring.
Asotus, a foolish spendthrift who longs to become a courtier and a master

In [20]:
tokenizer.decode(inp.input_ids[0])

'WHO NORMALLY DELIVERS THE OPENING PROLOGUE IN THE PLAY?</s> The play begins with three pages disputing over the black cloak usually worn by the actor who delivers the prologue. They draw lots for the cloak, and one of the losers, Anaides, starts telling the audience what happens in the play to come; the others try to suppress him, interrupting him and putting their hands over his mouth. Soon they are fighting over the cloak and criticizing the author and the spectators as well. In the play proper, the goddess Diana, also called Cynthia, has ordained a "solemn revels" in the valley of Gargaphie in Greece. The gods Cupid and Mercury appear, and they too start to argue. Mercury has awakened Echo, who weeps for Narcissus, and states that a drink from Narcissus\'s spring causes the drinkers to "Grow dotingly enamored of themselves." The courtiers and ladies assembled for the Cynthia\'s revels all drink from the spring. Asotus, a foolish spendthrift who longs to become a courtier and a mast

In [21]:
outputs = model.generate(**inp, max_new_tokens=100)

In [22]:
tokenizer.decode(outputs[0])

'<pad><extra_id_0>,<extra_id_1>,<extra_id_2> the cloak, and one of the losers, Anaides, starts telling the audience what happens in the play to come. Soon they are fighting over the cloak and criticizing the author and the spectators as well.</s>'